In [3]:
import ollama
def ask_ollama(prompt):
    response = ollama.chat(model='mistral', messages=[
    {
        'role': 'user',
        'content': f"{prompt}",
    },
    ])
    return response['message']['content']

In [13]:
def build_prompt(question, context):
    return f"""You are a helpful teaching assistant with knowledge of the YouTube playlist used in a course that teaches about LLMs (Large Language Models). Students ask you questions about where
    content is in the playlist as well as questions about the concepts and topics in the playlist. Answer the question using the information provided in the CONTEXT section below.
    If what they are asking does not appear in the CONTEXT provided. Please explicitly state that you were not able to locate the information in the video content and then provide
    an answer as best you can:

    QUESTION: {question}

    CONTEXT: {context}
    """

In [16]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

def retrieve_documents_question(query, index_name="transcripts", max_results=5):
    es = Elasticsearch("http://localhost:9200")
    
    # Generate query embedding
    query_embedding = model.encode(query)
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "should": [
                    {
                        "multi_match": {
                            "query": query,
                            "fields": ["Title", "Text^3"],
                            "type": "most_fields"
                        }
                    },
                    {
                        "script_score": {
                            "query": {"match_all": {}},
                            "script": {
                                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                                "params": {"query_vector": query_embedding}
                            }
                        }
                    }
                ]
            }
        },
        "_source": ["Title", "Text", "TimeStamp"],
        # "sort": [
        #     {"TimeStamp": {"order": "asc"}}
        # ]
    }
    
    try:
        response = es.search(index=index_name, body=search_query)
        result_docs = [hit['_source'] for hit in response['hits']['hits']]
        return result_docs
    except Exception as e:
        print(f"Error retrieving documents: {e}")
        return []

# Example usage
query = "When did we start discussing Mage in the course?"
# query = generate_embeddings(query)
results = retrieve_documents_question(query)

In [19]:
print(ask_ollama(build_prompt("When did we start discussing Mage in the course",results)))

 I could not locate any information about "Mage" in the provided video content for the course on LLMs (Large Language Models). It seems that the topics discussed in the playlist include RAG evaluation, monitoring and containerization, interface and ingestion pipeline, evaluating vector retrieval. If there is a specific lecture or module where Mage was mentioned, I would recommend double-checking the titles and timestamps of each video again, or reaching out to the course instructor for clarification.


In [18]:
print(build_prompt("When did we start discussing Mage in the course",results))

You are a helpful teaching assistant with knowledge of the YouTube playlist used in a course that teaches about LLMs (Large Language Models). Students ask you questions about where
    content is in the playlist as well as questions about the concepts and topics in the playlist. Answer the question using the information provided in the CONTEXT section below.
    If what they are asking does not appear in the CONTEXT provided. Please explicitly state that you were not able to locate the information in the video content and then provide
    an answer as best you can:

    QUESTION: When did we start discussing Mage in the course

    CONTEXT: [{'Title': 'llm zoomcamp 7.3 - rag evaluation', 'Text': ' the only thing we can do right now is as a judge of course when we start running the um when we start running the', 'TimeStamp': 128.28}, {'Title': 'llm zoomcamp 7.5 - monitoring and containerization', 'Text': ' all the data is there of course we can do it even more automated in a more automa

In [17]:
results

[{'Title': 'llm zoomcamp 7.3 - rag evaluation',
  'Text': ' the only thing we can do right now is as a judge of course when we start running the um when we start running the',
  'TimeStamp': 128.28},
 {'Title': 'llm zoomcamp 7.5 - monitoring and containerization',
  'Text': ' all the data is there of course we can do it even more automated in a more automated way so when we start the',
  'TimeStamp': 1151.72},
 {'Title': 'llm zoomcamp 7.4 - interface and ingestion pipeline',
  'Text': ' complex and um i think what we did uh in the course llm',
  'TimeStamp': 333.28},
 {'Title': 'llm zoomcamp 3.3.4 - evaluating vector retrieval',
  'Text': ' hi everyone in this series of videos we are discussing evaluating our retrieval methods we have already evaluated uh',
  'TimeStamp': 1.12},
 {'Title': 'llm zoomcamp 3.3.4 - evaluating vector retrieval',
  'Text': ' example so when we did it in the first module our yeah so our query was',
  'TimeStamp': 1675.36}]